In [1]:
import pandas as pd  
import numpy as np
import re
import random
from unidecode import unidecode 

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 
# Nuevas importaciones:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from time import sleep  

In [2]:
#! FUNCIÓN PARA EXTRAER Y LIMPIAR DATOS DE IMDb

def extraccion_detalles_imdb (archivo_csv = 'Ruta o nombre del archivo.csv', columna1 = 'Nombre columna títulos', columna2 = 'Nombre columna años' , columna3 = 'Nombre columna ID' , salida_detalles = "Ruta o nombre de salida_detalles.csv" , salida_actores = "Ruta o nombre de salida_actores.csv"):   
    
    """
    Función para extraer y limpiar datos de IMDb.

    Parámetros:
    archivo_csv (str): Ruta o nombre del archivo CSV que contiene los datos de las películas.
    columna1 (str): Nombre de la columna que contiene los títulos de las películas en el archivo CSV.
    columna2 (str): Nombre de la columna que contiene los años de las películas en el archivo CSV.
    salida_detalles (str): Ruta o nombre del archivo de salida donde se guardarán los detalles de las películas.
    salida_actores (str): Ruta o nombre del archivo de salida donde se guardarán los nombres de los actores.

    Retorna:
    - Una lista de tuplas que representan los detalles de las películas encontradas.
    - Una lista de nombres de actores encontrados en las películas.

    La función extrae datos de IMDb para cada película listada en el archivo CSV proporcionado, como la puntuación, director, guionista, argumento, duración y otros detalles relevantes. Luego guarda estos datos en un archivo CSV especificado y también guarda una lista de nombres de actores en otro archivo CSV.
    """
    
    #! ------------------------------------------------------------------------------ Abre el CSV
    
    # Carga el archivo CSV
    peliculas_df = pd.read_csv(archivo_csv)

    #! ------------------------------------------------------------------------------ Extrae los datos

    # Inicia el navegador Chrome
    driver = webdriver.Chrome()

    # Abre la página de IMDb
    driver.get("https://www.imdb.com/")
    driver.maximize_window()
    sleep(3)

    try:
        # Acepta las cookies si es necesario
        driver.find_element("css selector" , "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW").click()
    except:
        pass

    sleep(random.randint(1, 3))

    print("--- Empieza la extracción de datos ---")
    print(" ")

    lista_detalles_peliculas = []
    lista_actores = []
    ultimo_descanso_corto = 0
    ultimo_descanso_largo = 0

    for indice, pelicula in peliculas_df.iterrows():
        
        # Condición para asegurar que una vez que se haya hecho un descanso corto, no se repita hasta pasado un mínimo de iteraciones
        if indice >= ultimo_descanso_corto + random.randint(100 , 129):
            descanso_corto = random.randint(5, 20) # Escoge un número random y lo asigna a la variable descanso corto
        
            if indice != 0 and indice % descanso_corto == 0:
                ultimo_descanso_corto = indice 
                print(f"Hago un pequeño break. Estoy en la iteración número {indice}.")
                print(f"Volveré en unos 5-10 segundos")
                sleep(random.randint(5 , 10) + random.uniform(0.1 , 0.5))
                print(f"¡Ya estoy de vuelta! Sigo con lo mío")
                print(' ')
            
        # Condición para asegurar que una vez que se haya hecho un descanso largo, no se repita hasta pasado un mínimo de iteraciones
        if indice >= ultimo_descanso_largo + random.randint(530 , 550):
            descanso_largo = random.randint(5, 20) # Escoge un número random y lo asigna a la variable descanso largo
            
            # Si el número de iteración es divisible por el número de descanso, se para durante una cantidad de segundos aleatoria
            if indice != 0 and indice % descanso_largo == 0:
                tiempo_descanso = random.randint(60 , 120)
                ultimo_descanso_largo = indice 
                print(f"Paro a por un café ☕. Estoy en la iteración número {indice}.")
                print(f"Volveré en {round(tiempo_descanso/60 , 1)} minutos")
                sleep(tiempo_descanso)
                print(f"¡Ya estoy de vuelta! Sigo con lo mío")
                print(' ')
        
        nombre_peli = pelicula[columna1].lower()  # Obtiene el nombre de la película
        anio_peli = int(pelicula[columna2])  # Obtiene el año de la película
        id_peli = pelicula[columna3]  # Define la frase de búsqueda
        encontrada = False
        
        # Establece variables de inicio
        puntuacion_encontrada = False
        reparto_encontrado = False
        duracion_encontrada = False
        argumento_encontrado = False
        guion_encontrado = False
        direccion_encontrada = False
        datos_reparto = "-"
        intentos = 5

        # Intenta realizar la búsqueda en IMDb
        try:  
            # Localiza el elemento de la barra de búsqueda
            search_bar = driver.find_element("xpath", '//*[@id="suggestion-search"]')

            # Simula acciones de usuario: hacer clic en la barra de búsqueda y escribir texto
            ActionChains(driver).click(search_bar).send_keys(id_peli).perform()

            # Espera un breve período de tiempo después de escribir el texto
            sleep(random.randint(1, 2) + random.uniform(0.1, 0.5))

            # Localiza y hace clic en el botón de búsqueda
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="suggestion-search-button"]')))
                
            search_button.click()
                   
        # Si encuentra una página con error 404 al acceder a la barra de búsqueda, vuelve a la home                                                                 
        except:
            print("No se encuentra la barra de búsqueda")
            print(" ")
            sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))
            try:
                try: 
                    driver.find_element('css selector', '#error > div.error_attrib > a').click()
                    sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))
                    print(f"¡Vaya! Me han pillado, pero no pasa nada: Acabo de sortear el error 404 🥷🏻")
                    print(" ")
                except:
                    driver.find_element('css selector', '#error > div.error_message > a').click()
                    sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5)) 
                    print(f"¡Vaya! Me han pillado, pero no pasa nada: Acabo de sortear el error 404 🥷🏻")
                    print(" ")
            except:
                pass
    
            sleep(random.randint(3 , 4) + random.uniform(0.1 , 0.5)) 
            
            info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text   
            info_separada = info.split('\n')
            
    #! ------------------------------------------------------------------------------ Limpia y almacena los datos        
            
            # Itera por los elementos de la cabecera
            for indice , elemento in enumerate(info_separada):
                
                patron_horas = r'\b(\d+)min\b'
                
                if 'direccion' == unidecode(elemento.lower()):
                    datos_direccion = info_separada[indice+1]
                    direccion_encontrada = True
                    
                    # Extrae todas las coincidencias de nombre y apellidos de la string
                    totales_d = re.findall('[A-Z][a-z]*', datos_direccion)
                    
                    # Si hay más de 1 nombre y 1 apellido, entonces guarda en una lista la combinación de todos ellos
                    if len(totales_d) > 2 and len(totales_d) % 2 == 0:
                        
                        datos_direccion = [f"{totales_d[i]} {totales_d[i+1]}" for i in range(0, len(totales_d), 2)]  
                        
                    # Si la lista de nombres y apellidos es impar, combina todos los elementos excepto el último y añade el último al penúltimo    
                    elif len(totales_d) > 2 and len(totales_d) % 2 != 0:
                        
                        datos_direccion = [f"{totales_d[i]} {totales_d[i+1]}" for i in range(0, len(totales_d) - 2, 2)]
                        
                        # Agrega el último elemento al penúltimo
                        datos_direccion[-1] += f" {totales_d[-1]}"                       
                
                if 'guion' == unidecode(elemento.lower()):
                    datos_guion = info_separada[indice+1]
                    guion_encontrado = True
                    
                    # Extrae todas las coincidencias de nombre y apellidos de la string
                    totales_g = re.findall('[A-Z][a-z]*', datos_guion)
                    
                    # Si hay más de 1 nombre y 1 apellido, entonces guarda en una lista la combinación de todos ellos
                    if len(totales_g) > 2 and len(totales_g) % 2 == 0:
                        
                        datos_guion = [f"{totales_g[i]} {totales_g[i+1]}" for i in range(0, len(totales_g), 2)]
                        
                    # Si la lista de nombres y apellidos es impar, combina todos los elementos excepto el último y añade el último al penúltimo
                    elif len(totales_g) > 2 and len(totales_g) % 2 != 0:
                        
                        datos_guion = [f"{totales_g[i]} {totales_g[i+1]}" for i in range(0, len(totales_g) - 2, 2)]
                        
                        # Agrega el último elemento al penúltimo
                        datos_guion[-1] += f" {totales_g[-1]}"
                    
                if 'argumento' in elemento.lower() and unidecode(info_separada[indice+1].lower()) != 'direccion' and unidecode(info_separada[indice+1].lower()) != 'guion':
                    argumento = info_separada[indice+1]
                    argumento_encontrado = True
                    
                if 'puntuacion en imdb' == unidecode(elemento.lower()):
                    puntuacion = info_separada[indice+1]
                    puntuacion = float(puntuacion.replace(',' , '.'))
                    puntuacion_encontrada = True
                
                if 'reparto' in elemento.lower():
                    reparto_encontrado = True
                    
                if 'puntuacion' in elemento and re.search(patron_horas , info_separada[indice-1]):
                    horas = info_separada[indice-1]   
                    duracion_encontrada = True
                                
                    if len(horas.split()) > 1:
                        duracion_minutos = int(horas.split()[0].replace('h' , ''))*60 + int(horas.split()[1].replace('min', ''))
                    
                    else:
                        duracion_minutos = int(horas.replace('min', '')) 
            
            # Si no había datos de puntuación, asigna '-' por defecto            
            if not puntuacion_encontrada:
                puntuacion = '-'
            
            # Si no había datos de duración, asigna '-' por defecto     
            if not duracion_encontrada:
                duracion_minutos = '-' 
            
            # Si no había datos de argumento, asigna '-' por defecto     
            if not argumento_encontrado:
                argumento = '-'
                
            # Si no había datos de guion, asigna '-' por defecto     
            if not guion_encontrado:
                datos_guion = '-'
                
            # Si no había datos de direccion, asigna '-' por defecto     
            if not direccion_encontrada:
                datos_direccion = '-'
            
            # Si encontró datos del reparto en el resumen de la película, intenta extraerlos de la página
            if reparto_encontrado:
                
                try:
                    
                    if 'reparto' in driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div[1]').text.lower():
                        
                        # Extrae los datos del reparto principal
                        datos_reparto = driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div[2]').text
                        
                        # Separa los datos obtenidos y se queda sólo con los nombres de los actores
                        datos_reparto = datos_reparto.split('\n')[::2] 
                        
                        # Si la longitud de la lista de actores obtenida es superior a 10, la limitamos 
                        if len(datos_reparto) > 10:
                            datos_reparto = datos_reparto[:10]
                    
                    elif 'reparto' in driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[1]').text.lower():
                        
                        # Extrae los datos del reparto principal
                        datos_reparto = driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]').text
                        
                        # Separa los datos obtenidos y se queda sólo con los nombres de los actores
                        datos_reparto = datos_reparto.split('\n')[::2] 
                        
                        # Si la longitud de la lista de actores obtenida es superior a 10, la limitamos 
                        if len(datos_reparto) > 10:
                            datos_reparto = datos_reparto[:10]  
                            
                    elif 'reparto' in driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[1]').text.lower():
                        
                        # Extrae los datos del reparto principal
                        datos_reparto = driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]').text
                        
                        # Separa los datos obtenidos y se queda sólo con los nombres de los actores
                        datos_reparto = datos_reparto.split('\n')[::2] 
                        
                        # Si la longitud de la lista de actores obtenida es superior a 10, la limitamos 
                        if len(datos_reparto) > 10:
                            datos_reparto = datos_reparto[:10]       
                
                except:
                    datos_reparto = '-'
                    
            # Cambia la variable a positiva, ya que ha encontrado la película en IMDB
            encontrada = True
        
        # Si encontró la película en IMDB:
        if encontrada:
            # Crea una tupla con todos los detalles de la película
            tupla_peli = (puntuacion , datos_direccion , datos_guion , argumento , duracion_minutos , nombre_peli , anio_peli)
            
            # Añade la tupla al listado de películas
            lista_detalles_peliculas.append(tupla_peli)
            
            # Añade el listado de actores a la lista en caso de haberlos encontrado
            if reparto_encontrado and datos_reparto != '-':
                lista_actores.extend(datos_reparto)
        
        # Si no la encontró, crea una tupla con resultados vacíos y la peli    
        else:
            tupla_peli = (np.nan , np.nan , np.nan , np.nan , np.nan , nombre_peli , anio_peli)
            
            # Añade la tupla al listado de películas
            lista_detalles_peliculas.append(tupla_peli)
        
        sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))    
    
    # Convierte la lista de actores a set para que elimine los duplicados y luego vuelve a pasarlo a lista
    lista_actores = list(set(lista_actores))
    
    #! ------------------------------------------------------------------------------ Guarda los archivos
    
    try:
        df_lista_detalles_peliculas = pd.DataFrame(lista_detalles_peliculas)
        df_lista_detalles_peliculas.to_csv(salida_detalles)
        print(f"Los detalles de las películas se han guardado en:")
        print(f"-{salida_detalles} ✅")
    except:
        print(f"Ha habido un error al intentar guardar el archivo con los detalles de las películas")
        print(f"Prueba a usar la variable 'detalles_pelis' para guardarlo de forma manual")
        print(" ") 
        pass
    
    try:
        df_lista_actores = pd.DataFrame(lista_actores)
        df_lista_actores.to_csv(salida_actores)
        print(f"Los detalles de las películas se han guardado en:")
        print(f"-{salida_actores} ✅")
    
    except:
        print(f"Ha habido un error al intentar guardar el archivo con los actores de las películas")
        print(f"Prueba a usar la variable 'lista_actores' para guardarlo de forma manual")
        print(" ") 
        pass
    
    return lista_detalles_peliculas , lista_actores     


#! FUNCIÓN PARA GUARDAR DE FORMA MANUAL

def guardar_csv (dict_json):
    
    """
    Función para guardar un diccionario en un archivo CSV de forma manual.

    Parámetros:
    dict_json (dict): Diccionario de datos a ser guardados en un archivo CSV.

    Retorna:
    None

    La función toma un diccionario y lo guarda en un archivo CSV especificado por el usuario. Solicita al usuario que ingrese el nombre o la ruta del archivo CSV donde se guardarán los datos.
    """
    
    df_dict_json = pd.DataFrame(dict_json)
    
    df_dict_json.to_csv(input("Indica el nombre o ruta del archivo. Recuerda poner .csv al final"))  

In [ ]:
# Lista de variables a modificar según archivo:

ruta_csv_a_extraer = 'Accion\cortos_accion_2000_2004.csv' # Nombre o ruta del archivo del que queremos extraer información
columna_titulos = '1' # Nombre de la columna en la que se encuentran los títulos de las pelis
columna_anios = '2' # Nombre de la columna en la que se encuentran los años de las pelis
columna_id = '4' # Nombre de la columna en la que se encuentran las IDs de las pelis
ruta_salida_detalles_pelis = 'Accion\detalles_cortos_accion_2000_2004.csv' # Nombre o ruta del archivo en el que guardaremos los detalles de las películas
ruta_salida_actores_pelis = 'Accion\_actores_cortos_accion_2000_2004.csv' # Nombre o ruta del archivo en el que guardaremos los actores de las películas


# Llamada a la función (no modificar):

detalles_pelis , lista_actores = extraccion_detalles_imdb(ruta_csv_a_extraer, columna_titulos, columna_anios, columna_id, ruta_salida_detalles_pelis, ruta_salida_actores_pelis)

# SI LA FUNCIÓN DA ERROR: 
# -Si los archivos no se encuentra en la misma ubicación que este jupyter, tendremos que darle la ruta. P.e: en lugar de 'pelis_drama_2000_2004.csv' >>> 'Drama/pelis_drama_2000_2004.csv'
# -Hay que sustituir '1' y '2' por el nombre de las columnas en las que aparezca el título de la película y el año
# --- Por ejemplo: '1' >>> 'nombre' y '2' >>> 'aÃ±o estreno'
# --- Podemos saber el nombre de las columnas abriendo el archivo csv en un excel y mirando en la celda A1

In [ ]:
# En caso de que haya habido un error al guardar los archivos de forma automática, usar esta llamada de función para guardarlos de forma manual:

guardar_csv(detalles_pelis) 

In [ ]:
# En caso de que haya habido un error al guardar los archivos de forma automática, usar esta llamada de función para guardarlos de forma manual:

guardar_csv(lista_actores)